# Prediction Pipelines with `article_relevance`

The Article relevance tool was designed for use with the Neotoma Paleoecology Database to produce a workflow that allows users to supply an updatable list of DOIs from publications. This list of DOIs is then used to extract metadata from [CrossRef](https://crossref.org) that can be constructed into a list of text embeddings from which we can develop predictive models.  The tooling allows us to generate multiple predictive models, along with the ability to perform a grid-search for optimal hyper-parameter tuning for all models.

The workflow provides predictive outputs as to whether an article might be suited for inclusion into a research database, a probability estimate for the prediction, as well as time-stamped predictions and the ability of a user to override the prediction. In this way we can test model evolution, and provide the opportunity for curated stewardship of model predictions.

In [1]:
import os
from dotenv import load_dotenv
import src.article_relevance as ar

load_dotenv()

True

This package includes multiple requirements for proper use. All requirements are placed in an `requirements.txt` file.

In addition, at minimum, we require a file with properly labelled DOI data. This data should contain the following columns:

* doi: A properly formatted DOI, with only the shoulder and endpoint. e.g., `10.5467/22343.whatever`
* label: A categorical label that can be used to identify whether or not the article is suitable for inclusion into the database.

This project contains two data resources in the `data` folder:

* `raw/neotoma_crossref.csv`: data directly exported from Neotoma (publication is already included in Neotoma)
* `raw/labelled_data.csv`: manually labelled data for model training

Our first goal is to import the data and add the required CrossRef metadata. To do that we build one of several data objects. We are storing data in memory, but using an AWS S3 bucket to maintain data consistency.

In [2]:
DOI_STORE = {'Bucket':os.environ['S3_BUCKET'],'Key':'doi_store.parquet'}
METADATA_STORE = {'Bucket':os.environ['S3_BUCKET'],'Key':'metadata_store.parquet'}
EMBEDDING_STORE = {'Bucket':os.environ['S3_BUCKET'],'Key':'embedding_store.parquet'}
PREDICTION_STORE = {'Bucket':os.environ['S3_BUCKET'],'Key':'prediction_store.parquet'}
LABELLING_STORE =  {'Bucket':os.environ['S3_BUCKET'],'Key':'labelling_store.parquet'}

These objects represent the file elements we'll be working with for the predictive models.

## Loading Raw Data

For our purposes we want to identify both the data source and the labelling. For any object we want to know the data source and certainty with which it was labelled. Data from our source should be most trustworthy, labelled data should have some indication of the labeller, unlabelled data should reflect that fact.

The first set of data we load is raw data from the database itself (`db_data`), and a set of labelled data (`labelled_data.csv`). The labelled data was all labelled by the same person in this case ("Simon Goring"), so when we insert the data to the `LABELLING_STORE` document we'll make sure to add that.

DOIs get entered in many ways by users. Incorrect data entry may result in DOIs that do not properly resolve, or are simply incorrect. Additionaly data entry errors may result in leading or trailing whitespace. To help us ensure that data is entered correctly we will use the function `ar.clean_dois()`. This function returns a `dict` with the `clean` and `removed` DOIs submitted by the user. This helps support data cleaning down the road.

In [3]:
import pandas as pd
from datetime import datetime

db_data = pd.read_csv('data/raw/neotoma_dois.csv')
label_data = pd.read_csv('data/raw/labelled_data.csv')

all_doi = set(db_data['doi'].tolist() + label_data['doi'].tolist())
doi_set = ar.clean_dois(all_doi)

submitted_dois = pd.DataFrame({'doi':list(doi_set['clean']), 'date': datetime.now()})

print(f'A total of {len(db_data.index) + len(label_data.index)} DOIs were submitted.')
print(f'Of those objects there were {len(all_doi)} unique DOIs.')
print(f'There were {len(doi_set.get("clean"))} unique and valid DOIs.')

A total of 8918 DOIs were submitted.
Of those objects there were 2700 unique DOIs.
There were 2657 unique and valid DOIs.


The `submitted_dois` now has all of our valid and unique DOIs, labelled with a timestamp for the date/time they were added. This is a seperate document from the full metadata markup of these elements, and we'll further separate the dois, and metadata all from the embeddings later on.

We do object separation in part to reduce our file input/output overhead. The workflow assumes that all DOIs in this set of records stem from this core `doi_store` record. We keep a `datetime` tag so that we can assess updates and model outcomes against the available data at the time of model building or prediction.

The `push_s3` method gives us several options, including `check` and `create`. This allows us to create new objects in the S3 bucket if the Bucket/Key combination in the `s3_object` parameter does not yet exist, and the `check` flag allows us to validate our objects before we push them up to the cloud.

### First Run

#### Upload the raw DOI data

The first time we run this workflow we can assume that the data does not exist in the cloud, so we'll use `push_s3()`. This will push our full set of DOIs into the DOI_STORE bucket. At this point if we look in our S3 Bucket we will see a file called `doi_store.parquet` (from the `DOI_STORE` variable) in our bucket. The `push_s3` function allows us to `create` the file if it does not exist, and to simply `check` two file/DataFrame versions against each other.

In [4]:
ar.push_s3(s3_object = DOI_STORE, pa_object = submitted_dois, check = False, create = True)

In the future, if we are adding new records to the complete set of DOIs we can use the function `update_dois()`. This will accept a vector of DOI strings, validate the DOIs, concatenate them with the list in `DOI_STORE` and update `DOI_STORE` with the new DOIs and their timestamps:

In [5]:
new_dois = ['10.1590/s0102-69922012000200010', '10.1090/S0002-9939-2012-11404-2', '10.1063/1.4742131', '10.1007/s13355-012-0130-x']

ar.update_dois(s3_object = DOI_STORE, dois = new_dois)

4 DOIs submitted.
4 DOIs valid.
2657 DOI(s) already submitted.
4 DOIs valid and previously unseen.
-- 2661 total DOIs.


,doi,date
0,10.1111/nph.17627,2024-06-03 12:02:23.266687
1,10.1073/pnas.1914211117,2024-06-03 12:02:23.266687
2,10.1016/j.purol.2019.08.262,2024-06-03 12:02:23.266687
3,10.1016/j.immuni.2023.02.013,2024-06-03 12:02:23.266687
4,10.1080/03639045.2023.2183724,2024-06-03 12:02:23.266687
...,...,...
2656,10.1039/d3fo01016c,2024-06-03 12:02:23.266687
2657,10.1590/s0102-69922012000200010,2024-06-03 12:02:37.058148
2658,10.1090/S0002-9939-2012-11404-2,2024-06-03 12:02:37.058375
2659,10.1063/1.4742131,2024-06-03 12:02:37.058588


#### Get Article Metadata

Article metadata will be stored in the object `METADATA_STORE`. It will likely be a large object, and so we want to reduce the amount of time we pass data back and forth, and we want to make sure it is synced with the object stored in `DOI_STORE` so that we know what articles we've accessed/processed and so that we have a guide for future labelling.

To get article metadata we want to use the `crossref_query()` function. We have to pass in both the `DOI_STORE` and the `METADATA_STORE`. If there are records that are in the `DOI_STORE` but not in the `METADATA_STORE` then we will poll CrossRef for the metadata. We can also check to see if there are any records in `METADATA_STORE` that aren't in `DOI_STORE`. This should not happen, but we can always check to be sure.

In [6]:
metadata = ar.crossref_query(DOI_STORE, METADATA_STORE, create = True)

A total of 2661 DOIs submitted.
Of the total there are 2661 valid DOIs.
DOI metadata exists for 2661 records.
Fetching DOI metadata for 0 records.
Pulled metadata file from S3 Bucket.
Pulling metadata for 213 records.


From here we have an object in the cloud with DOIs and object metadata where it is available. The output `DataFrame` contains several key metadata fields from CrossRef, including `title` and `subject`, that we will use later in analysis. We also append a field `data`, to indicate the date on which a record was accessed, and `valid`, as to whether or not a record was available from CrossRef. We can then pass the `metadata` `DataFrame` into an embedding model to create a numeric matrix that can be used for predictive modeling.

## Building Embeddings

Before we build embeddings we need to define which fields we are to use to generate the embeddings. We choose to use the article title and abstract (when available), all subject labels and the journal title. We treat the journal title and subjects with one-hot-encoding. This may have limitations with "unseen" journals, however we can perform *post hoc* tests to examine the weight that these encodings have with regards to predictive probabilities.

To generate embeddings we also need to clean up the text, removing stopwords, punctuation and other elements. We use `data_preprocessing()` to clean up the text fields. We use `sklearn` for the prediction models with sentence embeddings for the text, and One-Hot-Encoding for journal subjects.

In [7]:
processed_data = ar.data_preprocessing(METADATA_STORE)

/home/simon/Documents/Neotoma/article-relevance/src/article_relevance/data_preprocessing.py:33: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_text = [BeautifulSoup(i, "lxml").text for i in text_batch]


We now have `processed_data`, which includes a column indicating whether or not the article is ready for building an embedding model, and if it has sufficient metadata for modeling.  We want to add embeddings and elements for OneHotEncoding to support prediction:

In [9]:
embeddings = ar.add_embeddings(processed_data, 'titleSubtitleAbstract', embedding_store = EMBEDDING_STORE)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

No new objects to be embedded.


Once we've added the embeddings to all our data, we need to pull out the "ground truthed" labels to actually decide whether something is appropriately a Neotoma-relevant paper or not. Given an existing "label_store" in our S3 bucket, we can build a set of labelled data.

Here we already have `label_data` (as identified above). the `ar.add_labels()` function pulls down any existing records from S3 and appends to them. If there is no existing set of labelled data it will create a new file.

We then pull the set of data from Neotoma (the `db_data` object) and add it into the set of labelled data. Since we know these records come from Neotoma, we can label them all as "Neotoma-related" documents.

This system is built to allow for uncertainty in labelling, for example, we can allow for one person to identify an article as "Neotoma-related" and a second person to identify it as "not-Neotoma", however, we assume that the "source" (i.e., SJG, or SDV) will only have one authoritative assignment, and that will always be the most recent assignment.

Thus, we can always add "old" data to the label store, so long as it is properly date stamped and identifies the source of the labels.

In [12]:
first_labels = ar.add_labels(LABELLING_STORE, label_data, create = True)

neotoma_labels = pd.DataFrame([db_data.doi]).transpose().drop_duplicates()
neotoma_labels['label'] = 'Neotoma'
neotoma_labels['source'] = 'Neotoma'
all_labels = ar.add_labels(LABELLING_STORE, neotoma_labels, create = True)

Now we want to merge all our data sources into a single Pandas data-frame called `data_model`, to begin the actual classification and validation. After this operation we'll have a data frame that has the columns `doi`, `valid`, `subject`, and then a list of all the embedding scores (n = 767).

In [18]:

dois = ar.pull_s3(DOI_STORE)
metadata = ar.pull_s3(METADATA_STORE).loc[metadata['valid'] == True]
embeddings = ar.pull_s3(EMBEDDING_STORE)
labels = ar.pull_s3(LABELLING_STORE)

data_model = pd.merge(dois,
                      metadata,
                      how = 'inner',
                      on = ['doi'])[['doi', 'valid', 'subject']].merge(embeddings.drop('date', axis = 1),
                                                                       on = ['doi'],
                                                                       how = 'inner')


Next, we'll bring in the labelled data, using a `target` column to change our classification system to a direct binary classification problem. Here we assume false positives (the "maybe-Neotoma" class) aren't a huge problem.

In [19]:
labels.loc[labels['label'] == 'Neotoma', 'target'] = 1
labels.loc[labels['label'] == 'Maybe Neotoma', 'target'] = 1
labels.loc[pd.isna(labels['target']), 'target'] = 0
labels = labels.drop(['label', 'data', 'source'], axis = 1)

data_model = data_model.merge(labels, on = ['doi'], how = 'right')
data_model = data_model.loc[data_model['valid'] == True].drop(['valid','date'], axis = 1)

Once we've put the data together, we want to be able to split it into testing and training datasets. We can use the `sklearn` package to do this:

In [20]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_model.copy(),
                                                    data_model['target'],
                                                    test_size = 0.2,
                                                    random_state = 42)
neotoma_encoder = ar.NeotomaOneHotEncoder(min_count = 3)
X_encoded = neotoma_encoder.fit_transform(X_train[['subject']])

We want to run a set of different classifiers to determine the appropriate classification method. To do this we define a set of parameters for each of the different classification models we want to test. We provide these parameters through a set of lists to support a grid-search approach. This workflow is designed to use classifiers from the `sklearn` package.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

classifiers = [
        (LogisticRegression(max_iter=1000), {
            'C': [0.001, 0.01, 0.1, 1, 10],
            'max_iter': [100, 1000, 10000],
            'penalty': ['l2'],
          'solver': ['liblinear', 'lbfgs']
        }),
        (DecisionTreeClassifier(class_weight="balanced"), {
            'max_depth': range(10, 100, 10)
        }),
        (KNeighborsClassifier(weights='uniform', algorithm='auto'), {
            'n_neighbors': range(5, 100, 10)
        }),
        (BernoulliNB(binarize=0.0), {
            'alpha': [0.001, 0.01, 0.1, 1.0]
        }),
        (RandomForestClassifier(), {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30]
        })
    ]

Once we have defined the different classifiers, we can run the model training:

In [21]:
import json

resultsDict = ar.relevancePredictTrain(x_train = X_train,
                                       y_train = y_train,
                                       classifiers = classifiers)
with open('results.json', 'w', encoding='UTF-8') as f:
    json.dump(resultsDict, f)

NameError: name 'classifiers' is not defined

This then results in several outputs. The first is the object `resultsDict`, which is a dictionary with each model name, the model pipeline, and then a set of summary statistics, including the total run time, the training and test recall, F1, precision and accuracy.

In addition, the final model `joblib` files are put into a `models` folder in the parent directory. This allows us to access and run these models in the future. The model files are all time-stamped, so that we can also examine changes in model performance over time and archive the model outputs.